In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt

## ALL

[OK] Wrote /home/moshtasa/Research/phd-svd-recsys/SVD/Book/result/rec/top_re/0929/SVD/figure/Adult/Adult_unique_totals.txt and /home/moshtasa/Research/phd-svd-recsys/SVD/Book/result/rec/top_re/0929/SVD/figure/Adult/Adult_unique_totals.png
[OK] Wrote /home/moshtasa/Research/phd-svd-recsys/SVD/Book/result/rec/top_re/0929/SVD/figure/Adventure/Adventure_unique_totals.txt and /home/moshtasa/Research/phd-svd-recsys/SVD/Book/result/rec/top_re/0929/SVD/figure/Adventure/Adventure_unique_totals.png
[OK] Wrote /home/moshtasa/Research/phd-svd-recsys/SVD/Book/result/rec/top_re/0929/SVD/figure/Children_s/Children_s_unique_totals.txt and /home/moshtasa/Research/phd-svd-recsys/SVD/Book/result/rec/top_re/0929/SVD/figure/Children_s/Children_s_unique_totals.png
[OK] Wrote /home/moshtasa/Research/phd-svd-recsys/SVD/Book/result/rec/top_re/0929/SVD/figure/Classics/Classics_unique_totals.txt and /home/moshtasa/Research/phd-svd-recsys/SVD/Book/result/rec/top_re/0929/SVD/figure/Classics/Classics_unique_totals.